# Data Engineer Certification - Practical Exam - Supplement Experiments

1001-Experiments makes personalized supplements tailored to individual health needs.

1001-Experiments aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at 1001-Experiments plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage. 

This integration helps 1001-Experiments provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

1001-Experiments currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [ ]:
# Use as many python cells as you wish to write your code

In [7]:
# Step 1. Import the necessary librarires
import pandas as pd
import numpy as np

In [8]:
# Step 2. Load all datasets with date parsing
health_df = pd.read_csv('user_health_data.csv', parse_dates=['date'])
supplement_df = pd.read_csv('supplement_usage.csv', parse_dates=['date'])
experiments_df = pd.read_csv('experiments.csv')
profiles_df = pd.read_csv('user_profiles.csv')

In [9]:
# Define age group categories
def categorize_age(age):
    if pd.isna(age):
        return 'Unknown'
    age = int(age)
    if age < 18:
        return 'Under 18'
    elif age <= 25:
        return '18-25'
    elif age <= 35:
        return '26-35'
    elif age <= 45:
        return '36-45'
    elif age <= 55:
        return '46-55'
    elif age <= 65:
        return '56-65'
    else:
        return 'Over 65'

# Apply age grouping
profiles_df['user_age_group'] = profiles_df['age'].apply(categorize_age)

# Keep only required columns
profiles_df = profiles_df[['user_id', 'email', 'user_age_group']]


In [10]:
# Convert dosage to grams
def convert_to_grams(row):
    if pd.isna(row['dosage']):
        return np.nan
    if row['dosage_unit'].lower() == 'mg':
        return row['dosage'] / 1000
    elif row['dosage_unit'].lower() == 'g':
        return row['dosage']
    else:
        return np.nan

# Apply conversion
supplement_df['dosage_grams'] = supplement_df.apply(convert_to_grams, axis=1)

In [11]:
# Join experiments metadata
supplement_df = supplement_df.merge(
    experiments_df[['experiment_id', 'name']],
    on='experiment_id',
    how='left'
)

# Rename for clarity
supplement_df.rename(columns={'name': 'experiment_name'}, inplace=True)

In [12]:
# Outer merge on user_id and date
merged_df = pd.merge(
    health_df,
    supplement_df,
    on=['user_id', 'date'],
    how='outer'
)

In [13]:
# Join with user profile data
merged_df = merged_df.merge(profiles_df, on='user_id', how='left')

In [14]:
# Fill supplement_name where missing
merged_df['supplement_name'] = merged_df['supplement_name'].fillna('No intake')

# Ensure optional columns exist
for col in ['dosage_grams', 'is_placebo', 'experiment_name']:
    if col not in merged_df.columns:
        merged_df[col] = np.nan

In [16]:
# Final column order
final_columns = [
    'user_id',
    'date',
    'email',
    'user_age_group',
    'experiment_name',
    'supplement_name',
    'dosage_grams',
    'is_placebo',
    'average_heart_rate',
    'average_glucose',
    'sleep_hours',
    'activity_level'
]

# Subset and format
merged_df = merged_df[final_columns]
merged_df['date'] = pd.to_datetime(merged_df['date'])

# Show preview
merged_df.head()

,user_id,date,email,user_age_group,experiment_name,supplement_name,dosage_grams,is_placebo,average_heart_rate,average_glucose,sleep_hours,activity_level
0,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-01-31,hi_1@example.com,Over 65,Sleep Quality,Zinc,0.149315,False,93.055612,70.089910,8.8h,1
1,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-02-28,hi_1@example.com,Over 65,NaN,No intake,NaN,NaN,88.059964,78.411148,8.0H,3
2,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-03-31,hi_1@example.com,Over 65,Endurance,Magnesium,0.239949,True,78.373746,107.418818,11.9h,1
3,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-03-31,hi_1@example.com,Over 65,Focus,Placebo,0.416022,False,78.373746,107.418818,11.9h,1
4,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-04-30,hi_1@example.com,Over 65,Recovery,Placebo,0.324913,False,62.204061,117.259092,5.1h,1


In [17]:
import pandas as pd
import numpy as np

def merge_all_data(health_path, supplement_path, experiments_path, profiles_path):
    # Load CSVs
    health_df = pd.read_csv(health_path, parse_dates=['date'])
    supplement_df = pd.read_csv(supplement_path, parse_dates=['date'])
    experiments_df = pd.read_csv(experiments_path)
    profiles_df = pd.read_csv(profiles_path)

    # Merge supplement_usage with experiments
    supplement_df = supplement_df.merge(experiments_df, on='experiment_id', how='left')

    # Handle missing supplement info
    supplement_df['supplement_name'] = supplement_df['supplement_name'].fillna('No intake')
    supplement_df['dosage_grams'] = np.where(
        supplement_df['dosage_unit'] == 'mg',
        supplement_df['dosage'] / 1000,
        supplement_df['dosage']
    )
    supplement_df['dosage_grams'] = supplement_df['dosage_grams'].where(supplement_df['supplement_name'] != 'No intake')

    # Merge health + supplement
    merged_df = pd.merge(health_df, supplement_df, on=['user_id', 'date'], how='outer')

    # Merge user profile
    merged_df = merged_df.merge(profiles_df, on='user_id', how='left')

    # Format age group
    def get_age_group(age):
        if pd.isnull(age): return 'Unknown'
        age = int(age)
        if age < 18: return 'Under 18'
        elif age <= 25: return '18-25'
        elif age <= 35: return '26-35'
        elif age <= 45: return '36-45'
        elif age <= 55: return '46-55'
        elif age <= 65: return '56-65'
        else: return 'Over 65'
    
    merged_df['user_age_group'] = merged_df['age'].apply(get_age_group)

    # Fill missing supplement info
    merged_df['supplement_name'] = merged_df['supplement_name'].fillna('No intake')
    merged_df['experiment_name'] = merged_df['name']  # rename for clarity

    # Select final columns
    final_columns = [
        'user_id',
        'date',
        'email',
        'user_age_group',
        'experiment_name',
        'supplement_name',
        'dosage_grams',
        'is_placebo',
        'average_heart_rate',
        'average_glucose',
        'sleep_hours',
        'activity_level'
    ]
    
    # Ensure all expected columns exist
    for col in final_columns:
        if col not in merged_df:
            merged_df[col] = np.nan

    return merged_df[final_columns]


In [18]:
df = merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')
df.head()

,user_id,date,email,user_age_group,experiment_name,supplement_name,dosage_grams,is_placebo,average_heart_rate,average_glucose,sleep_hours,activity_level
0,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-01-31,hi_1@example.com,Over 65,Sleep Quality,Zinc,0.149315,False,93.055612,70.089910,8.8h,1
1,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-02-28,hi_1@example.com,Over 65,NaN,No intake,NaN,NaN,88.059964,78.411148,8.0H,3
2,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-03-31,hi_1@example.com,Over 65,Endurance,Magnesium,0.239949,True,78.373746,107.418818,11.9h,1
3,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-03-31,hi_1@example.com,Over 65,Focus,Placebo,0.416022,False,78.373746,107.418818,11.9h,1
4,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-04-30,hi_1@example.com,Over 65,Recovery,Placebo,0.324913,False,62.204061,117.259092,5.1h,1
